In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#import warnings
#warnings.filterwarnings('ignore')
# import numpy as np

# pd_jupiter = pd.read_excel('./jupyter.xlsx', engine='openpyxl')
# pd_X = pd_jupiter[['x1', 'x2', 'x3']]
# pd_Y = pd_jupiter.y
# train_X, test_X, train_Y, test_y = train_test_split(pd_X, pd_Y, test_size=0.1, random_state=42)

# treeCLF = RandomForestClassifier(random_state=0, min_samples_split=2, min_samples_leaf=1)
# parameters = {
#     'n_estimators': range(1, 5),
#     'max_depth': range(3, 5),
# }
# cv = [(slice(None), slice(None))]

# searchTree = GridSearchCV(treeCLF, parameters, cv=cv)
# searchTree.fit(train_X, train_Y)
# bestEstimatorTree = searchTree.best_estimator_
# pred_y = bestEstimatorTree.predict(test_X)
# test_y = test_y.reset_index(drop=True)
# resultDatasetPredictions = pd.DataFrame({
#     'test_y': test_y,
#     'pred_y': pred_y,
#     'diff': pred_y - test_y,
# })
# # resultDatasetPredictions
# joblib.dump(bestEstimatorTree, './jupyter/jupyter.pkl')
#pd_jupiter = pd.read_csv('./jupyter.csv')
#pd_jupiter.iloc[3]

In [2]:



# and later you can load it
# clf = joblib.load('filename.pkl')

# Рабочее пространство



## Загрузка exel

In [3]:
#Очистка Датафрейма
def ClearDF(df, parameters):
    return df[parameters]

In [4]:
#Переименование параметров
def renameDF(df, parameters, rename_parameters):
    for i in range(len(parameters)):
        df.rename(columns={parameters[i]: rename_parameters[i]}, inplace=True)
    return df

In [5]:
#Группировка параметров по дате
def groupDF(df, group_parameter):
    df_copy = df.copy()
    df_copy[group_parameter] = pd.to_datetime(df[group_parameter]).dt.date
    df_copy = df_copy.groupby(group_parameter).sum()
    return df_copy

In [6]:
#Cоздание ДатаФрейма для группировки данных по интервалам дня(утро, день, вечер, ночь)
def createDfinterval(intervals, parameters):
    df = pd.DataFrame({'interval': intervals})
    #Заполнение ДатаФрейма нулями
    for i in range(len(parameters)):
        df[parameters[i]] = [0] * 5
    return df

In [7]:
#Группировка параметров по времени суток
def paddingDF(df, df_xlsx, date, parameters):
    for i in range(df_xlsx.shape[0]):
        #Получение чсисла равное часам в данной записи
        hour = df_xlsx[date][i].hour
        if((hour>= 0 and hour < 6) or (hour>= 22 and hour < 24)) :
            for j in range(len(parameters)):
                df[parameters[j]][0] += df_xlsx[parameters[j]][i]                                                                                               
        if(hour>= 6 and hour < 10):
            for j in range(len(parameters)):
                df[parameters[j]][1] += df_xlsx[parameters[j]][i]
        if(hour>= 10 and hour < 14):
            for j in range(len(parameters)):
                df[parameters[j]][2] += df_xlsx[parameters[j]][i]
        if(hour>= 14 and hour < 18):
            for j in range(len(parameters)):
                df[parameters[j]][3] += df_xlsx[parameters[j]][i]
        if(hour>= 18 and hour < 22):
            for j in range(len(parameters)):
                df[parameters[j]][4] += df_xlsx[parameters[j]][i]
        return df

In [8]:
#Группировка по признаку
def groupDFcsv(df, group_parameter, parameters):
    df_parameter = ClearDF(df, parameters)
    df_parameter = df_parameter.groupby(group_parameter).sum()
    return df_parameter

In [9]:
#ТГ
def tg_full():
    #Создание ДатаФраймов 
    dep_xlsx_tg = pd.read_excel("jupyter/dep_tg.xlsx")
    
    #Исследуемые параметры
    parameters_tg = ["Views","Text","VR","Date","Media 1"]
        
    #Название параметров
    rename_parameters_tg = ['views','title','coef_ohv','datetime_post','media']
    
    #Исследуемые параметры
    investigated_parameters_tg = ['datetime_post','views','coef_ohv']
    
    #Исследуемые параметры без учета времени
    investigated_parameters_tg_group = ['views','coef_ohv']
    
    #Переменная времени(индекс)
    date = 'datetime_post'
    
    #Интервалы времени
    interval = ['night', 'morning', 'snack', 'lunch', 'dinner']
    
    
    #Вызов функции очистки
    dep_xlsx_tg = ClearDF(dep_xlsx_tg, parameters_tg)
    
    #Вызов функции переименовывания параметров
    dep_xlsx_tg = renameDF(dep_xlsx_tg, parameters_tg, rename_parameters_tg)
    
    #Вызов функции группировки по параметру datetime_post
    dep_xlsx_tg_date = groupDF(dep_xlsx_tg,'datetime_post')
    
    #Фызуов функции создания Датафрейма для группировки по интервалам
    dep_xlsx_tg_date_interval = createDfinterval(interval, investigated_parameters_tg_group)
    
    #Фузов функции заполнения ДатаФрейма
    dep_xlsx_tg_date_interval = paddingDF(dep_xlsx_tg_date_interval, dep_xlsx_tg, date, investigated_parameters_tg_group)
        
    return dep_xlsx_tg_date, dep_xlsx_tg_date_interval

In [10]:
#ВК
def vk_full():
    #Создание ДатаФраймов 
    dep_xlsx_vk = pd.read_excel("jupyter/dep_vk.xlsx")
    
    #Исследуемые параметры
    parameters_vk = ["Likes","Reposts","Comments","Views","ER","Text","VR","Date","Media 1"]
    
    #Название параметров
    rename_parameters_vk = ['likes','reposts','comments','views','coef_vov','title','coef_ohv','datetime_post','media']
    
    #Исследуемые параметры
    investigated_parameters_vk = ['datetime_post','likes','comments','views','reposts','coef_vov','coef_ohv']
    
    #Исследуемые параметры без учета времени
    investigated_parameters_vk_group = ['likes','comments','views','reposts','coef_vov','coef_ohv']
    
    #Переменная времени(индекс)
    date = 'datetime_post'
    
    #Интервалы времени
    interval = ['night', 'morning', 'snack', 'lunch', 'dinner']
    
    
    #Вызов функции очистки
    dep_xlsx_vk = ClearDF(dep_xlsx_vk, parameters_vk)
    
    #Вызов функции переименовывания параметров
    dep_xlsx_vk = renameDF(dep_xlsx_vk, parameters_vk, rename_parameters_vk)
    
    #Вызов функции группировки по параметру datetime_post
    dep_xlsx_vk_date = groupDF(dep_xlsx_vk,'datetime_post')
    
    #Фызуов функции создания Датафрейма для группировки по интервалам
    dep_xlsx_vk_date_interval = createDfinterval(interval, investigated_parameters_vk_group)
    
    #Фузов функции заполнения ДатаФрейма
    dep_xlsx_vk_date_interval = paddingDF(dep_xlsx_vk_date_interval, dep_xlsx_vk, date, investigated_parameters_vk_group)
        
    return dep_xlsx_vk_date, dep_xlsx_vk_date_interval

In [11]:
#CSV
def csv_full():
    #Создание ДатаФраймов 
    dep_csv = pd.read_csv("jupyter/dep_user.csv")
    
    #Исследуемые параметры
    parameters_csv_gender = ['gender', 'contract_sum', "purchase_sum", "purchase_count"]
    parameters_csv_city = ['city', 'contract_sum', "purchase_sum", "purchase_count"]
    parameters_csv_birth_date = ['birth_date', 'contract_sum', "purchase_sum", "purchase_count"]
    
    #Вызов функций для группировки по признаку
    #Группировка по гендеру
    dep_csv_gender = groupDFcsv(dep_csv, 'gender',parameters_csv_gender)
    #Группировка по городу
    dep_csv_city = groupDFcsv(dep_csv, 'city',parameters_csv_city)
    #Группировка по дате рождения
    dep_csv_birth_date = groupDFcsv(dep_csv, 'birth_date', parameters_csv_birth_date)
        
    return dep_csv_gender, dep_csv_city, dep_csv_birth_date